In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error

from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import ElasticNet
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression

In [2]:
# df = pd.read_csv('hotData.csv')
df = pd.read_csv('hotoutliers.csv')


In [3]:
y = df['results']
X = df.drop(columns=['results'])

X_train, X_test, y_train, y_test = train_test_split(
     X, y, test_size=0.10, random_state=2)

In [4]:
def cross_val(model):
    pred = cross_val_score(model, X_train, y_train, cv=10)
    return pred.mean()

def print_evaluate(true, predicted):  
    mae = metrics.mean_absolute_error(true, predicted)
    mse = metrics.mean_squared_error(true, predicted)
    rmse = np.sqrt(metrics.mean_squared_error(true, predicted))
    r2_square = metrics.r2_score(true, predicted)
    print('MAE:', mae)
    print('MSE:', mse)
    print('RMSE:', rmse)
    print('R2 Square', r2_square)
    print('__________________________________')
    
def evaluate(true, predicted):
    mae = metrics.mean_absolute_error(true, predicted)
    mse = metrics.mean_squared_error(true, predicted)
    rmse = np.sqrt(metrics.mean_squared_error(true, predicted))
    r2_square = metrics.r2_score(true, predicted)
    return mae, mse, rmse, r2_square

In [5]:
model = ElasticNet(alpha=0, tol=10)

In [6]:
model.fit(X_train, y_train)

/var/folders/j_/555m2zps099832fjh_m8jjnc0000gn/T/ipykernel_31641/180087699.py:1: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  model.fit(X_train, y_train)
/Users/ryantalbot/opt/anaconda3/envs/tf2/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:648: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(


ElasticNet(alpha=0, tol=10)

In [7]:
test_pred = model.predict(X_test)
train_pred = model.predict(X_train)

In [8]:
results_df = pd.DataFrame(data=[["Linear Regression", *evaluate(y_test, test_pred) , cross_val(LinearRegression())]], 
                          columns=['Model', 'MAE', 'MSE', 'RMSE', 'R2 Square', "Cross Validation"])

In [9]:
print('MAPE: '+str(mean_absolute_percentage_error(y_test, test_pred)))
# print('MAPE: '+str(mean_absolute_percentage_error(y_train, train_pred)))
# alpha=0, tol=10 MAPE: 0.4381142743608524


MAPE: 0.5391731926724728


In [10]:
# mape: 1.4778
# alpha=0 MAPE: 0.5071599152907355


In [11]:
# MAPE: 1.4778202775570333
# alpha=0 MAPE: 0.5071599152907355
#  alpha = 0 is equivalent to an ordinary least square, 
#     solved by the LinearRegression object.

In [12]:
print('ELASTIC NET')
print('Test set evaluation:\n_____________________________________')
print_evaluate(y_test, test_pred)
print('====================================')

results_df = pd.DataFrame(data=[["Elastic Net Regression", *evaluate(y_test, test_pred) , cross_val(ElasticNet())]], 
                            columns=['Model', 'MAE', 'MSE', 'RMSE', 'R2 Square', "Cross Validation"])
results_df = results_df.append(results_df, ignore_index=True)

ELASTIC NET
Test set evaluation:
_____________________________________
MAE: 4.236562107097874
MSE: 28.14697193161663
RMSE: 5.305371988052923
R2 Square 0.3807812735689651
__________________________________


In [13]:
data = {'y_test': y_test, 'y_pred': test_pred}
new_df = pd.DataFrame(data)
diff = (new_df['y_test'] - new_df['y_pred']).abs()
new_df['diff'] = diff
new_df = new_df.sort_values(by='diff', ascending=True)


In [14]:
new_df

,y_test,y_pred,diff
217,8,7.995175,0.004825
66,12,12.023773,0.023773
41,15,15.095115,0.095115
134,21,21.132687,0.132687
3,6,6.248125,0.248125
10,6,7.733135,1.733135
250,13,11.122790,1.877210
245,7,9.395178,2.395178
94,12,14.535987,2.535987
140,14,11.434895,2.565105


In [15]:
new_df['diff'].mean()

4.236562107097875

In [16]:
new_df['diff'].median()

3.304418162159024